In [1]:
from rdflib.plugins.stores import sparqlstore

In [2]:
import rdflib
from rdflib import URIRef, Literal, Graph, Dataset

import loader
import discourse
import pandas as pd
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
from IPython.core.display import HTML


def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()

    display(Image(png))

def t2rdflibg(triples): #triples to rdflib graph
    g = Graph()
    for t in triples:
        g.add(t)
    return g


def nan2None(value):
    if pd.isnull(value) :
        return None
    else:
        return value


def get_data_rows(filename):
    data_rows_df = pd.read_csv(filename)
    data_rows=[]
    for i,r in data_rows_df.iterrows():
        r_d = {k:nan2None(v) for k,v in dict(r).items()}

        data_rows.append(r_d)
    del data_rows_df
    return data_rows

serial_key = { "Business Domain" : "ModelDomain", 
               "Model" : "Model", 
               "ModelType" : "ModelType", 
               "Entity" : "Class", 
               "Attribute" : "Attribute", 
               "DataType" : "DataType", 
               "Context" : "Context", 
               "SubContext" : "SubContext", 
               "Relationship" : "Relationship", 
               "Entity From" : "FromClass",
               "Attribute From" : "FromAttribute", 
               "Cardinality From" : "FromCardinality",
               "Entity To" : "ToClass",
               "Attribute To" : "ToAttribute",
               "Cardinality To" : "ToCardinality"
}



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#RelationshipFromClass belongs to more than one entity types: [owl.ObjectProperty, datamodels_rdf.MetaProperty, datamodels_rdf.RelationshipProperties]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#RelationshipToClass belongs to more than one entity types: [owl.ObjectProperty, datamodels_rdf.MetaProperty, datamodels_rdf.RelationshipProperties]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#Contains belongs to more than one entity types: [owl.ObjectProperty, owl.ObjectProperty, owl.ObjectProperty, datamodels_rdf.MetaProperty]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://www.semanticw

datamodels_rdf.Mapping_Attribute [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_AttributeHasDataType [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_Class [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_ClassContainsAttribute [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_Context [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_ContextContainsClass [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_ContextContainsSubContext [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_DomainContainsModel [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_FromAttribute [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_FromClass [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_Model [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_ModelContainsClass [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_ModelContainsContext [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_ModelContainsRelationship [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_ModelDomain [datamodels_rdf.Mapping]
datamodels_rdf.Mapping_ModelHasModelT

* Owlready2 * HermiT took 0.6517000198364258 seconds
* Owlready * Reparenting datamodels_rdf.RelationshipFromAttribute: {datamodels_rdf.RelationshipProperties, owl.ObjectProperty} => {datamodels_rdf.RelationshipProperties, datamodels_rdf.MetaProperty}
* Owlready * Reparenting datamodels_rdf.RelationshipToClass: {datamodels_rdf.RelationshipProperties, owl.ObjectProperty} => {datamodels_rdf.RelationshipProperties, datamodels_rdf.MetaProperty}
* Owlready * Reparenting datamodels_rdf.RelationshipToAttribute: {datamodels_rdf.RelationshipProperties, owl.ObjectProperty} => {datamodels_rdf.RelationshipProperties, datamodels_rdf.MetaProperty}
* Owlready * Reparenting datamodels_rdf.RelationshipToOptionality: {datamodels_rdf.RelationshipProperties, owl.ObjectProperty} => {datamodels_rdf.RelationshipProperties}
* Owlready * Reparenting datamodels_rdf.RelationshipFromOptionality: {datamodels_rdf.RelationshipProperties, owl.ObjectProperty} => {datamodels_rdf.RelationshipProperties}
* Owlready * Rep

In [3]:
jena = sparqlstore.SPARQLUpdateStore("http://localhost:3030/models/query",context_aware=True)
jena.open(("http://localhost:3030/models/query", "http://localhost:3030/models/update"))

In [4]:
# Create a Dataset
ds = Dataset(store=jena, default_union=True)
# Define a named graph within the dataset, this will contain all nodes contained within the graph referenced, or none if it doesn't exist yet.
g = ds.graph(URIRef("http://datamodels.raw"))


In [5]:
qr = g.query("""select ?s ?p ?o where { ?s ?p ?o. }""")

In [6]:
[r for r in qr]

[(rdflib.term.URIRef('http://this.com/this'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.Literal('another test')),
 (rdflib.term.URIRef('http://this.com/this'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.Literal('test'))]

In [7]:
g.add((URIRef("http://this.com/this"), RDF.type, Literal("test")))
g.add((URIRef("http://this.com/this"), RDF.type, Literal("another test")))

In [8]:
qr = g.query("""select ?s ?p ?o where { ?s ?p ?o. }""")

In [9]:
qrs=list([r for r in qr])
qrs[-5:]

[(rdflib.term.URIRef('http://this.com/this'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.Literal('another test')),
 (rdflib.term.URIRef('http://this.com/this'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.Literal('test'))]

In [10]:
# job_def is in the form (Serialization, key, data )
job_def_load_entities = ("Model Load", loader.onto.EntityAttributeRecord , serial_key, get_data_rows("model_entities.csv")  )
batch_manifest = [job_def_load_entities]

In [ ]:
graph = loader.spawn_rdflib_graph()